# Using music21 to parse MIDI data

In [1]:
from music21 import *

Let's start with a piece Ave Maria...

In [2]:
sBach = converter.parse("MIDI_Archive/Classical_Greatest/Bach/AveMaria.mid")
# sBach.show() #for this to work, you have to install a software called musescore

We can also play it!

In [3]:
sBach.show('midi')

# Extract MIDI sequence

To make a network, we want to extract the MIDI sequence for different parts. After piece is parsed, you can access different parts using piece.parts, and for each part p, p.notesAndRests would access the notes (or chords, aka notes happening together) and rests in the sequence, and secondsMap will provide the information of not only the note pitch, but also when it starts and when it ends.

However, there are rests that is not legit (those who didn't actually show in the sheet music) and notes with duration 0. We also probably want to keep track of very long rest, since that means a new sentence has begun. Therefore, we do a second round on the music piece, in order to filter notes that is shorter than a threhold and mark rest longer than a threshold.

In [2]:
def quantify_music(piece):
    from collections import defaultdict
    # first pass of the music, not removing notes and rests that is not reasonable
    midi_dict = {}
    time_dict = {}
    key = piece.analyze('key').name # tell us the key of the piece (eg. F major, C minor)
    duration_notes = []
    for i, p in enumerate(piece.parts):
        part_midi = []
        time = []
        for n in p.flat.notesAndRests.secondsMap: # secondsMap would include not only the note, but also the time information
            start = n['offsetSeconds']
            end = n['endTimeSeconds']
            time.append((start, end))
            element = n['element']
            try:
                part_midi.append(element.pitch.midi)
                if (end-start)!=0.0:
                    duration_notes.append((end-start))
            except:
                try:
                    part_midi.append([item.midi for item in element.pitches])
                except:
                    part_midi.append(128) # coding rest as 128
        midi_dict[i] = part_midi
        time_dict[i] = time
    # Indentify short rest and long rest
    midi_dict_prune = defaultdict(list)
    time_dict_prune = defaultdict(list)
    min_threshold = min(duration_notes)
    max_threshold = max(duration_notes)
#     print(min_threshold, max_threshold)
    for p in time_dict:
        for note, (start, end) in zip(midi_dict[p], time_dict[p]):
            if note!=128:
                midi_dict_prune[p].append(note)
                time_dict_prune[p].append((start, end))
            else:
                if end-start >= min_threshold:
                    if end-start > max_threshold:
                        midi_dict_prune[p].append(129)
                        time_dict_prune[p].append((start, end))
                    else:
                        midi_dict_prune[p].append(128)
                        time_dict_prune[p].append((start, end))
    return key, midi_dict_prune, time_dict_prune

In [5]:
sMozart = converter.parse("MIDI_Archive/Classical_Greatest/Mozart/K527 Overture ''Don Giovanni''.mid")

KeyboardInterrupt: 

In [6]:
%timeit key, midi_dict, time_dict = quantify_music(sMozart)

2.05 s ± 319 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
sBeatles = converter.parse("MIDI_Archive/Beatles/Beatles_Blackbird.mid")

In [7]:
key, midi_dict, time_dict = quantify_music(sBeatles)

In [8]:
len(midi_dict)

6

# Extract the whole corpus

## Get all the midi file name under this directory (including subdirectory)

In [3]:
import os
from glob import glob
import random
midis = []
start_dir1 = "../../dataset/midi/"
#start_dir2 = "MIDI_Archive/Jazz"
#start_dir3 = "MIDI_Archive/Metal_Rock"
#start_dir4 = "MIDI_Archive/american_folk"

def walk_dir(start_dir):
    midis = []
    pattern   = "*.mid"
    for dir,_,_ in os.walk(start_dir):
        midis.extend(glob(os.path.join(dir,pattern))) 
    return midis

midis = walk_dir(start_dir1)
#midis = walk_dir(start_dir1) + walk_dir(start_dir2) + walk_dir(start_dir3) + walk_dir(start_dir4)
#midis = random.sample(walk_dir(start_dir1), 10)
# random.sample(walk_dir(start_dir2), 50) + 
# random.sample(walk_dir(start_dir3), 50) + 
# random.sample(walk_dir(start_dir4), 50)

In [4]:
len(midis)

45129

In [ ]:
import signal
import json
midi_corpus = {}
count = 0
ind = 0
failed_midi = []
big_midi = []
class TimeoutException(Exception):   # Custom exception class
    pass

def timeout_handler(signum, frame):   # Custom signal handler
    raise TimeoutException

# Change the behavior of SIGALRM
signal.signal(signal.SIGALRM, timeout_handler)

for midi in midis:
    # Start the timer. Once 10 seconds are over, a SIGALRM signal is sent.
    signal.alarm(30)    
    # This try/except loop ensures that 
    #   you'll catch TimeoutException when it's sent.
    try:
        try:
        #if True:
            piece = converter.parse(midi)
            key, midi_dict, duration_dict = quantify_music(piece)
            #midi_corpus[midi.split("MIDI_Archive/")[1].split('.mid')[0].replace('/',' ')] = [key, midi_dict, duration_dict]
            fname = midi.split('/')[-1]
            fname = fname.split('.')[0]
            midi_corpus[fname] = [key, midi_dict, duration_dict]
            count+=1
        except:
            print("failed:",midi)
            failed_midi.append(midi)
    except TimeoutException:
        print("big:",midi)
        big_midi.append(midi)
        signal.alarm(0)
        continue # continue the for loop if function A takes more than 5 second
    else:
        # Reset the alarm
        signal.alarm(0)
    if count == 1000:        
        json.dump(midi_corpus, open("../../dataset/corpus/midi_corpus_%s.json"%ind,'w'),indent = 4)
        midi_corpus = {}
        count = 0
        ind += 1

if len(midi_corupus) > 0:
    json.dump(midi_corpus, open("../../dataset/corpus/midi_corpus_%s.json"%ind,'w'),indent = 4)

failed: ../../dataset/midi/69ab8697eb22b3c3dd0439d7546382fe.mid
failed: ../../dataset/midi/321d1c5f7afe08aaf196d2d0f23164a8.mid
failed: ../../dataset/midi/cccdc75525a7962269b0973c1ed993dd.mid
failed: ../../dataset/midi/23f7af1f08bc322d66556831265a556f.mid
failed: ../../dataset/midi/e58d485bcad6ef75f5cbc2826aa23170.mid
failed: ../../dataset/midi/93cb6e40788e60d65a89045394d3a42d.mid
failed: ../../dataset/midi/e0027ee77ffdbe7293f47a4690792c84.mid
failed: ../../dataset/midi/449e5619fe20213165bc57af74755d8c.mid
failed: ../../dataset/midi/e8824ac3b15d9588a53eb779a54a50dd.mid
failed: ../../dataset/midi/7e111ba414653c3fe7278045f8c181fb.mid
failed: ../../dataset/midi/361d6a1e62a298d29bfd32529daed630.mid
failed: ../../dataset/midi/10d393fe16bc9c600312b3dae16e7725.mid
failed: ../../dataset/midi/90a6c2bf6cd22c93f85d688ed45f7594.mid
failed: ../../dataset/midi/71225f8f0c59f72c9aec9ced11452bf3.mid
failed: ../../dataset/midi/f55e5e66b1f270716a7d8ecf56e8cf0a.mid
failed: ../../dataset/midi/ceb116de1d5cc

failed: ../../dataset/midi/3a0274ba6e15ad94d7d704f91e328cfa.mid
failed: ../../dataset/midi/7977cbeff218ddd7ce3c8c28cb874745.mid
failed: ../../dataset/midi/09b570666304556185ef165abdf20935.mid
failed: ../../dataset/midi/9e3fee144fe42ada397730236525abbc.mid
failed: ../../dataset/midi/f3a751f74510abdc1af33439f723a2fd.mid
failed: ../../dataset/midi/d619440d116ca9aea67c6470300e48b3.mid
failed: ../../dataset/midi/1c9bf67d1776639205a022234ff17d02.mid
failed: ../../dataset/midi/2e29b675bde5aa76bb60977dfa0d0180.mid
failed: ../../dataset/midi/bea02ea515ab602016d2e6892798f249.mid
failed: ../../dataset/midi/4ce893e4c7dcfb82eb8d669e26dc8c6d.mid
failed: ../../dataset/midi/05845b57a77a2352d0699dcfb645b5b9.mid
failed: ../../dataset/midi/5d491e364e160becbc9f7dc6cf12b041.mid
failed: ../../dataset/midi/36618f53ae43208fa30a782437dc1005.mid
failed: ../../dataset/midi/188849c22aefa406346ec6679362e315.mid
failed: ../../dataset/midi/3ceef52df82d38cf3501c432fb555f97.mid
failed: ../../dataset/midi/0e57e04e0fd3f

failed: ../../dataset/midi/ef9e8b8016d5a8808bab1c020f15285e.mid
failed: ../../dataset/midi/4a272022119865ae1dbf3cf27c95b0cb.mid
failed: ../../dataset/midi/ef46455e4bb38268081a391f9bf108c8.mid
failed: ../../dataset/midi/57f67513b3062bb7f50a678fdc1f1415.mid
failed: ../../dataset/midi/c1570bd149d3b8e63e45be58552fea19.mid
failed: ../../dataset/midi/6512e71682534de6e0b91214fe7e89ac.mid
failed: ../../dataset/midi/69fae6de095ba114a36d358a9cc3a0d6.mid
failed: ../../dataset/midi/cd6457386e34fc4390c82f2c5f8fa926.mid
failed: ../../dataset/midi/08a5abf41e967a927c3879a1d3118748.mid
failed: ../../dataset/midi/eaee238d0028ae7b9b908f52d61c700c.mid
failed: ../../dataset/midi/54e6518fe3dcddaadc578443549cad0e.mid
failed: ../../dataset/midi/19c44c4f769df56b9bf6f1f2a3a99f57.mid
failed: ../../dataset/midi/68691418bec5983b34c7ba7a4b72704e.mid
failed: ../../dataset/midi/e3291087df314fd30ea7d898e37e1182.mid
failed: ../../dataset/midi/cd830688ec25d0acd8c7627b21557e71.mid
failed: ../../dataset/midi/c79ac047fb1e5

failed: ../../dataset/midi/40970e679827bc654f21cca7493ccf4a.mid
failed: ../../dataset/midi/c05c31a6fe086d8e18826bffac772171.mid
failed: ../../dataset/midi/d7e9c6e5dcb3a59e41c7f0ef9640ec0d.mid
failed: ../../dataset/midi/50bd256584a24eea1eef9d7504464c0b.mid
failed: ../../dataset/midi/d64321774b0cf446505c9abcbd316c19.mid
failed: ../../dataset/midi/db1918aa33424f55e3b6a65b1e5ad6f1.mid
failed: ../../dataset/midi/ce75ad5b61a07c9813b81da63d5b9907.mid
failed: ../../dataset/midi/c168d981d8be33bfb389fcb8b906d3b8.mid
failed: ../../dataset/midi/2e6219ba16f631f3e41fa911a4e438de.mid
failed: ../../dataset/midi/f919ce2ee6a937d37158fe3783f17279.mid
failed: ../../dataset/midi/391098fcb2d61a916e9191c742a71bc7.mid
failed: ../../dataset/midi/54aca424ee84fbc4e4debaac107dfeca.mid
failed: ../../dataset/midi/7769a1797d53d02bb91e813244fb45fd.mid
failed: ../../dataset/midi/c5af9e99572e0c9ff2e5b3fa81cd2250.mid
failed: ../../dataset/midi/a359f07e88f17e579877d47f179fb69d.mid
failed: ../../dataset/midi/a51fa3cd297a8

failed: ../../dataset/midi/3affe2bce5f61f99cddb2a984c1dca2b.mid
failed: ../../dataset/midi/c9e5de6a747d14b12009bc6a957102bb.mid
failed: ../../dataset/midi/e90bb03f28c0bf8d5303de92b0ab172c.mid
failed: ../../dataset/midi/5e97d440f910be9e18ef2749e04c52ec.mid
failed: ../../dataset/midi/99f699bd106f3710c7620b72762f85af.mid
failed: ../../dataset/midi/5bd8f7d76f9ecea2bda8e8c9bf2ebae9.mid
failed: ../../dataset/midi/ae4cb652d07272e9ebc2462f66fb1385.mid
failed: ../../dataset/midi/c349645a37ced754e96d2ad486f7ade7.mid
failed: ../../dataset/midi/1160211b834b204c72dc32b93ffec13b.mid
failed: ../../dataset/midi/07be4faaa0e8fb9ab4d85eb39132e954.mid
failed: ../../dataset/midi/87018fda07571522cfd01b15186bf3be.mid
failed: ../../dataset/midi/9f56ecef1909736605549015cc1ed9cf.mid
failed: ../../dataset/midi/1d319be82d79da8732c0b86e3351b155.mid
failed: ../../dataset/midi/265eca348b8e7aab6e48adcd971d8523.mid
failed: ../../dataset/midi/f289ec40f73bd386efba8e7070b18614.mid
failed: ../../dataset/midi/840be5c7fc237

...

failed: ../../dataset/midi/52b1b235feffee10991dddc55a9f143b.mid
failed: ../../dataset/midi/62572f3b42f9aef2d5942dbbcb0aa572.mid
failed: ../../dataset/midi/caeaf46ed44f21ec15d037a16e50a32c.mid
failed: ../../dataset/midi/8f79f418c46b511534eef179b3ef0e73.mid
failed: ../../dataset/midi/d9f0ffff57f7a07ca485d6a1e139cc05.mid
failed: ../../dataset/midi/0936d1c2db0c04f0024309820cf6a537.mid
failed: ../../dataset/midi/09ecccd34acbbe01a0a438a314a4d35d.mid
failed: ../../dataset/midi/fe06fda67375940d5635fbf704685cfe.mid
failed: ../../dataset/midi/59ec5c91613c3afe80fca5b79b66cb89.mid
failed: ../../dataset/midi/a9be02ca5a9ee596c6e0e53054a41207.mid
failed: ../../dataset/midi/f1be134b947dfde3eece2861546ffd82.mid
failed: ../../dataset/midi/b11565d31424100a00493fffa64914f7.mid
failed: ../../dataset/midi/b71cbc1b4a55c9110bb8d6fbc0025057.mid
failed: ../../dataset/midi/12fedd05fd9d56639c106f2af577c8b0.mid
failed: ../../dataset/midi/97f1949243607aa32913a4b29a78f56b.mid
failed: ../../dataset/midi/7cafc0e0bcaa2

failed: ../../dataset/midi/aff195d5d6f4a04f1b7fea8797072a96.mid
failed: ../../dataset/midi/cd7f0689e7aca8c665f62bfff09616ad.mid
failed: ../../dataset/midi/a599f1b4ba63a967c920c223536b5e92.mid
failed: ../../dataset/midi/2b58b84e35d809b0fe3717bc344205df.mid
failed: ../../dataset/midi/00457b059c4cda82db2fd7d8eb7adfde.mid
failed: ../../dataset/midi/d33b6aae7eca53d654350ec8d5632df2.mid
failed: ../../dataset/midi/9295651989612defd2bddceefde3d241.mid
failed: ../../dataset/midi/ca1c7906ed80ade81206de8c2ebf1d89.mid
failed: ../../dataset/midi/e281f43a82f63105310bec30a32496a3.mid
failed: ../../dataset/midi/0ab8282f106ade01f1ad2f9777141145.mid
failed: ../../dataset/midi/8f7783be48f8c33cca2a3f18c75f6cdd.mid
failed: ../../dataset/midi/d8ba8b2f104d9b3c7c57764850c8a3c1.mid
failed: ../../dataset/midi/80e51d7b618f94fb16e1c45f0ffd3e32.mid
failed: ../../dataset/midi/19a4e50c8a3d4a1b3ce98827180f4ec0.mid
failed: ../../dataset/midi/89cc0d87a637868b8fcb19ef0ae47591.mid
failed: ../../dataset/midi/75aa9bfa965f0

failed: ../../dataset/midi/99d64f8d4626b7dbe48dddb2b56ff643.mid
failed: ../../dataset/midi/0914d3e06259339ab5e622715e72b6e9.mid
failed: ../../dataset/midi/cf658424eab9b0da113ce25db549387e.mid
failed: ../../dataset/midi/17e49ae6659a5765d91d3c416ef78567.mid
failed: ../../dataset/midi/b5cb9524f836652be52810996f1c049c.mid
failed: ../../dataset/midi/2a820c155fbebc1c9ecca7adf1452416.mid
failed: ../../dataset/midi/5efa9eafeb535386a4aa23efd2f4aa72.mid
failed: ../../dataset/midi/0b7b7c84a6361c36b7be0b6a85a5493c.mid
failed: ../../dataset/midi/4b1b3248c01d3e823ce9f79137a5fab1.mid
failed: ../../dataset/midi/56109b95836a857785fdd8150f28c609.mid
failed: ../../dataset/midi/b8e38b4ddabef3c223988f50581bb48d.mid
failed: ../../dataset/midi/e2d08748e907ec154ef83a610c3b0eee.mid
failed: ../../dataset/midi/62227bdcff2668fabe0d8e849d8762c0.mid
failed: ../../dataset/midi/9241e1e6139df7ea77aaa73ecd69a788.mid
failed: ../../dataset/midi/57a905c70d04d9b63a869cb82f040666.mid
failed: ../../dataset/midi/751f238b559a7

failed: ../../dataset/midi/c27ad72041e5a9741b4a49fa4ab9100e.mid
failed: ../../dataset/midi/c01ca30615912f7e1a49df3b53f23e63.mid
failed: ../../dataset/midi/6f798b9d8f8d761e78a2606cfe2418be.mid
failed: ../../dataset/midi/3fe470a84843a1f2f2c6105350912b52.mid
failed: ../../dataset/midi/f5e2b78459144cef807ea4d7779ca69c.mid
failed: ../../dataset/midi/fd588af7cd0c8205549a4cd51fd965e8.mid
failed: ../../dataset/midi/bb3c6ed5621b700dd26f3cb919c205b0.mid
failed: ../../dataset/midi/74f12be11403644449c70dd6dde1d18a.mid
failed: ../../dataset/midi/ad5373525ed3422aed264dccd534a221.mid
failed: ../../dataset/midi/173e7cd0873683fbc9113d719be5153a.mid
failed: ../../dataset/midi/311066677459cc981ecd673c10779559.mid
failed: ../../dataset/midi/eeac6ad2127d970743eb83dc054ceeae.mid
failed: ../../dataset/midi/0210eca054bb4fdcd80626b232ac345e.mid
failed: ../../dataset/midi/83bcfbf5c966c64badb67d7d27a574ba.mid
failed: ../../dataset/midi/eaf71de5728db0782b8b565073c9b4a8.mid
failed: ../../dataset/midi/d61fe6e78bd3a

failed: ../../dataset/midi/580f01e25932aa7a979699f301599235.mid
failed: ../../dataset/midi/df0f145dff074955d69be4a21f46c241.mid
failed: ../../dataset/midi/63fdfa4cd5461f885a1904714e0a4356.mid
failed: ../../dataset/midi/830d3ba7260f0e9d97ae98f04f1ee633.mid
failed: ../../dataset/midi/6439f32d376ab83385f23de634b9c7f4.mid
failed: ../../dataset/midi/c8bfc915c32da332191aa54854ac78f9.mid
failed: ../../dataset/midi/a0e6720aea419b036c8b999fff2884ae.mid
failed: ../../dataset/midi/8b86faa7bc628a8ddd445d4b865a588d.mid
failed: ../../dataset/midi/fa67cfde91cf53178c482c5555ab3645.mid
failed: ../../dataset/midi/528d6017f06126c157a07a976cca02f8.mid
failed: ../../dataset/midi/1a06f9bff7d197dac9342b9bd69c36ca.mid
failed: ../../dataset/midi/a5f05e5676cfa100060d8f6ca244e6a2.mid
failed: ../../dataset/midi/a50555092fea80c80c4573890531d197.mid


In [10]:
len(midi_corpus)

8090

# Simple network generation (one order)

In [11]:
import pickle
for each_piece in midi_corpus:
    from collections import Counter
    for each_part in midi_corpus[each_piece][1]:
        edges = []
        sequence = midi_corpus[each_piece][1][each_part]
        sequence = [item if type(item)!=list else max(item) for item in sequence]
        edges = list(zip(sequence[:-1], sequence[1:]))
        edges_with_weight = dict(Counter(edges))
        pickle.dump(edges_with_weight, open("Simple Network/%s %s.pickle"%(each_piece.replace('/',' '), each_part), "wb"))